In [ ]:
import numpy as np
import chainer
from chainer import cuda, Function, gradient_check, report, training, utils, Variable
from chainer import datasets, iterators, optimizers, serializers
from chainer import Link, Chain, ChainList
import chainer.functions as F
import chainer.links as L
from chainer.training import extensions
from tqdm import tqdm
import sys
import os
import matplotlib.pyplot as plt
import csv
import time
from IPython.display import HTML
from IPython.display import display

%matplotlib inline

# Machine Translation - Lab 2

**Neural Language Modeling**

In this lab, we will train a character level language model over an input text. We will also evaluate and sample (generate) text using the model.

## Installation and setup

We will be using Python3 and the Neural Network framework: [Chainer](http://chainer.org/)

To keep computation manageable, we will only use CPU(s) to train the models.


## Chainer

Try and work through the basic [chainer tutorial](http://docs.chainer.org/en/stable/tutorial/basic.html). You can skip *advanced* chainer concepts such as the [trainer class](http://docs.chainer.org/en/stable/tutorial/basic.html#trainer)

Finally, have a look at the multi-layer feedforward neural network [example on MNIST dataset](http://docs.chainer.org/en/stable/tutorial/basic.html#example-multi-layer-perceptron-on-mnist). The code for ```class MLP(Chain)``` definition shows how we can add [layers](http://docs.chainer.org/en/stable/reference/links.html#learnable-connections) (linear in this example, other choices include LTSM, GRU) and [activation functions](http://docs.chainer.org/en/stable/reference/functions.html#activation-functions) (RELU in this example, other choices include sigmoid, tanh ) in Chainer.

## character-level language model

### Background material
This lab is based on text and code from several online resources. It is highly recommended to check out the following:
- A great explanation and visualization of recurrent neural networks: [The Unreasonable Effectiveness of Recurrent Neural Networks](http://karpathy.github.io/2015/05/21/rnn-effectiveness/)
- [numpy based code to train a character-level language model](https://gist.github.com/karpathy/d4dee566867f8291f086)
- [Chainer implementation of the above source code](https://github.com/yusuketomoto/chainer-char-rnn)
- [Chainer reference](http://docs.chainer.org/en/stable/reference/)
- [Really cool explanation of LSTM (and GRU)](http://colah.github.io/posts/2015-08-Understanding-LSTMs/)

## Dataset

We will be using a dataset created by Andrej Karpathy by combining all of Shakespeare's work: [**tinyshakespeare**](https://github.com/karpathy/char-rnn/tree/master/data/tinyshakespeare) to train our model.

The input file: ```tinyshakespeare.txt``` can be found in the ```data/``` directory.



### Exploring the dataset

Let's look at the first few lines:

In [ ]:
print("------------- Shakespeare -------------")
!head -n 20 data/tinyshakespeare.txt
print("---------------------------------------")
print("\n------------- Polish -------------")
!head -n 20 data/pierwsza.txt
print("---------------------------------------")

You will notice that text is **structured**, i.e., there are speaker tags, and new line characters after each dialog. We leave everything intact including the **punctuation**.

As we will be **predicting** characters, let's explore the number of unique characters and their frequencies.

In [ ]:
prune_data_for_debugging = True

In [ ]:
def read_input_txt(fname):
    # define a dictionary. Each unique character will be a key in the dictionary
    # and the value will be its count
    chars = {}
    with open(fname, "r") as f:
        # read entire file content into buffer
        if prune_data_for_debugging:
            data = f.read()[:100000]
        else:
            data = f.read()
    # count occurrences of each character
    for c in data:
        if c not in chars:
            chars[c] = 1
        else:
            chars[c] += 1
    return data, chars

### Training dataset

In [ ]:
# define the training file location
# True : use English
# False: use Polish

if False:
    train_fname = os.path.join("data", "tinyshakespeare.txt")
    data_postfix = "shakespeare"
else:
    train_fname = os.path.join("data", "pierwsza.txt")
    data_postfix = "tadeusz"
    
data, chars = read_input_txt(train_fname)
vocab_size = len(chars.keys())
data_size = len(data)

# 90%-10% split for test and validation data
train_data = data[: data_size * 90 // 100]
dev_data = data[data_size * 90 // 100 :]
train_size = len(train_data)
dev_size = len(dev_data)

print("\n{0:^30}".format("data stats"))
print("{0:^30}".format("-" * len("data stats")))
print("{0:>20s} | {1:10d}".format("total characters", data_size))
print("{0:>20s} | {1:10d}".format("total unique chars", len(chars.keys())))
print("{0:>20s} | {1:10d}".format("train characters", train_size))
print("{0:>20s} | {1:10d}".format("test characters", dev_size))

# printing top 5 characters based on frequency

print("\n{0:^30}".format("top 5 characters in data"))
print("{0:^30}".format("-" * len("top 5 characters in data")))
print("{0:>20s} | {1:>10s}".format("char", "freq"))
out_rows = [(k, v) for k, v in sorted(list(chars.items()), reverse=True, key=lambda t:t[1])[:5]]
for k, v in out_rows:
    print("{0:>20s} | {1:>10d}".format(k, v))

### Preprocessing data

We perform minimal pre-processing of the dataset. This involves, for the purposes of efficiency, mapping each character to a unique integer id.

In [ ]:
char_to_ix = { ch:i for i,ch in enumerate(chars) }
ix_to_char = { i:ch for i,ch in enumerate(chars) }

In [ ]:
# prepare an integer representation of the dataset
data_char_to_ids = []
for c in data:
    data_char_to_ids.append(char_to_ix[c])
print("****original text excerpt**** \n{0:s}".format(data[:50]))
print("****text converted to integer representation**** \n{0:s}".format("".join(map(str, 
                                                                       data_char_to_ids[:50]))))

### Generating Shakespeare
The above table shows that the " " (space/blank) character occurs most often. As we train our language model, will the model learn proper word boundaries? That is, can the model predict when to insert a space between sequences?

We will answer this question using empirical observations by sampling our model to generate text.

[Spoiler - Shakespeare generated by Karpathy's model](http://cs.stanford.edu/people/karpathy/char-rnn/shakespear.txt)

## Training loss function

Cross-entropy loss is used to measure the total loss during the training phase.

## RNN models

We now create an RNN base class using LSTM.

Chainer reference can be found at: http://docs.chainer.org/en/stable/reference/

In [ ]:
class RNN(Chain):
    # constructor
    # vocab_size: indicates the number of unique characters in the vocabulary
    # n_units: size of the hidden layer
    # gpu_id: if >=0, use GPU, else CPU
    def __init__(self, vocab_size, n_units, gpu_id, use_LSTM=True):
        # initialise Chainer base class
        super(RNN, self).__init__()
        #---------------------------------------------------------
        # construct Neural Network
        #---------------------------------------------------------
        # add embedding layer
        # http://docs.chainer.org/en/stable/reference/links.html#embedid
        #---------------------------------------------------------
        self.add_link("embed", L.EmbedID(vocab_size, n_units))
        #---------------------------------------------------------
        # add LSTM or GRU layer
        if use_LSTM:
            # http://docs.chainer.org/en/stable/reference/links.html#lstm
            self.add_link("L1", L.LSTM(n_units, n_units))
        else:
            # http://docs.chainer.org/en/stable/reference/links.html#gru
            self.add_link("L1", L.StatefulGRU(n_units, n_units))
        '''
        ------------------------------------------------------------------
        Q3 - ADD CODE (around) HERE
        Go deep, add 1 more LSTM/GRU layer
        ------------------------------------------------------------------
        '''
        #---------------------------------------------------------
        # add output layer
        # http://docs.chainer.org/en/stable/reference/links.html#linear
        self.add_link("out", L.Linear(n_units,vocab_size))
        #---------------------------------------------------------
        
    def reset_state(self):
        # reset LSTM state
        # NOTE: the name field using during add_link call 
        # is used to refer to the layer
        self.L1.reset_state()
        
    # function to compute the forward pass through the network layers
    def forward(self, word):
        # lookup character embedding and compute the hidden state
        h1 = self.L1(self.embed(word))
        # compute the output layer over the hidden state
        out = self.out(h1)
        return out
    
    # function to compute the loss for training
    def __call__(self, c_n1, c_n2):
        # call forward to predict output
        # calculate softmax and then the cross entropy loss
        # Chainer (and most NN frameworks) provide functions to compute 
        # the softmax and cross entropy loss together
        self.loss = F.softmax_cross_entropy(self.forward(c_n1), c_n2)
        # return loss
        return self.loss
    

## Training code

### CPU/GPU selection

In [ ]:
# if >= 0, use GPU, if negative use CPU
gpuid = -1

# use cuda if GPU or numpy if CPU
xp = cuda.cupy if gpuid >= 0 else np

### Model parameters and setup

In [ ]:
def setup_model(model):
    #---------------------------------------------------------
    # copy model to GPU if selected
    #---------------------------------------------------------
    if gpuid >= 0:
        cuda.get_device(gpuid).use()
        model.to_gpu()
    #---------------------------------------------------------
    # optimizer
    # Select an optimizer
    # http://docs.chainer.org/en/stable/reference/optimizers.html
    # Alternatives: SGD, AdaGrad, RMSprop, etc
    # We can also add weight decay
    #---------------------------------------------------------
    optimizer = optimizers.Adam()
    # link optimizer to model
    optimizer.setup(model)
    # add weight decay
    optimizer.add_hook(chainer.optimizer.WeightDecay(rate=0.0005))
    
    return optimizer

## Set up model

In [ ]:
#---------------------------------------------------------
# define model
#---------------------------------------------------------
use_LSTM = False
lstm_postfix = "lstm" if use_LSTM else "gru"
model_baseline = RNN(vocab_size, 50, gpuid, use_LSTM=use_LSTM)
# the name field is used for logging stats and storing the model parameters
model_baseline.__dict__['name'] = "baseline_{0:s}_{1:s}".format(data_postfix, lstm_postfix)
'''
------------------------------------------------------------------
Q2 - ADD CODE (around) HERE

A. Define a new model using GRU instead of LSTM
B. Train and compute loss and perplexity as done for model_baseline
C. Compare the performance and training time between the GRU and LSTMs
------------------------------------------------------------------
'''
optimizer_baseline = setup_model(model_baseline)

### log training progress

### Function to train a batch of characters

In [ ]:
def train_batch(batch_data, model, optimizer):
    # reset LSTM initial state before training each batch
    model.reset_state()
    
    # reset loss
    loss = 0
    
    # for each character in the batch, the target character is the next in sequence
    for i, (curr_char, next_char) in enumerate(zip(batch_data, batch_data[1:]), start=1):
        # convert curr char into a chainer variable
        c1 = Variable(xp.asarray([curr_char], dtype=np.int32), volatile=False)
        c2 = Variable(xp.asarray([next_char], dtype=np.int32), volatile=False)
        # compute loss for each character and the prediction
        loss += model(c1, c2)
    
    # reset model gradients
    model.cleargrads()
    # compute loss through back prop
    loss.backward()
    # update parameters
    optimizer.update()
    return loss

### Function to sample from the model

In [ ]:
def sample(s_chars, n_chars, model):
    # Sample n_chars, with starting char: s_char
    sample_text = s_chars[:]
    
    model.reset_state()
    
    # initialize with s_char
    for s_char in s_chars:
        # compute the prediction from the starting character
        c = Variable(xp.asarray([char_to_ix[s_char]], dtype=np.int32), volatile=True)
        out = model.forward(c)

    prob = F.softmax(out)
    
    # Sample remaining characters
    for i in range(n_chars):
        # compute probability distribution over the characters using softmax
        prob = F.softmax(out)
        
        # if gpu, convert into numpy array in order to sample
        if gpuid >= 0:
            prob = cuda.to_cpu(prob.data)[0].astype(np.float64)
        else:
            prob = prob.data[0].astype(np.float64)
        
        # normalize probability
        prob /= np.sum(prob)

        # Sample next character from the predicted probability distribution
        index = np.random.choice(range(len(prob)), p=prob)

        # add sampled character to result
        sample_text.append(ix_to_char[index])
        
        c = Variable(xp.asarray([index], dtype=np.int32), volatile=True)
        # feed the character into the model
        out = model.forward(c)
        
    # combine sampled text into a string
    sampled_txt = ''.join(sample_text)
    return sampled_txt, prob

### main training function

In [ ]:
def train_loop(model, optimizer, data, n_epochs=3, batch_size=128, logging=True):
    if logging and model.__dict__['name']:
        train_log_fname = "{0:s}_train.log".format(model.__dict__['name'])
        train_log = open(train_log_fname, "w")
        train_writer = csv.writer(train_log, lineterminator="\n")
        train_writer.writerow(["iter", "loss"])
    
    # compute the number of batches in the data
    data_size = len(data)
    num_batches = data_size // batch_size
    
    # print reference text excerpt
    print("Reference text:\n{0:s}".format(data[1000:1100]))
    # sample text from the model
    # this is on the untrained model and the output will be random characters
    print("Sampled text:\n{0:s}".format(sample(data[1000], 100, model_baseline)))
    
    # start training epochs
    for epoch in range(n_epochs):
        # start progress bar for current epoch
        sys.stderr.flush()
        with tqdm(total=data_size) as pbar:
            sys.stderr.flush()
            for i in range(0, data_size, batch_size):
                # loop through the entire data in chunks of batch_size
                # note: using the integer id representation of the text
                batch_data = data_char_to_ids[i:i+batch_size]
                # call batch training
                loss = train_batch(batch_data, model, optimizer)
                # extract value of loss from Chainer variable returned
                loss = float(loss.data)
                # compute number of characters trained on so far
                it = (epoch * data_size) + i + batch_size
                # write loss to file
                train_writer.writerow([it, loss])
                '''
                ------------------------------------------------------------------
                Q1 - ADD CODE (around) HERE
                
                A. Compute perplexity and add this information to the train 
                log file
                B. Compute perplixity over validation data and create a new log file
                ------------------------------------------------------------------
                '''
                pbar.set_description("epoch={0:d}, loss={1:.6f}".format(epoch+1, loss))
                pbar.update(batch_size)
                # Sample 10 times
        # sample at the end of each epoch
        print("Sampling from starting char={0:s}, {1:d} characters".format(data[1000], 100))
        print("Sampled text:\n{0:s}".format(sample(data[1000], 100, model)))
    
    if train_log:
        train_log.close()
        print("train log file: {0:s}".format(train_log_fname))
    # Save model
    if model.__dict__['name']:
        serializers.save_npz("{0:s}.npz".format(model.__dict__['name']), model)
        print("model file: {0:s}".format(model.__dict__['name']))

In [ ]:
#---------------------------------------------------------
# training batch size
# compute loss for upto batch_size number of characters
#---------------------------------------------------------
batch_size = 128
#---------------------------------------------------------
# epochs
# to stop training
#--------------------------------------------------------- 
n_epochs = 1
#---------------------------------------------------------

In [ ]:
# train_loop(model_baseline, optimizer_baseline,
#            train_data, n_epochs=n_epochs, 
#            batch_size=batch_size, logging=True)

## Visualizing the model

In [ ]:
# load model
model_fname = "{0:s}.npz".format(model_baseline.__dict__['name'])
train_log_fname = "{0:s}_train.log".format(model_baseline.__dict__['name'])

serializers.load_npz(model_fname, model_baseline)
log_train = np.loadtxt(train_log_fname, delimiter=",", skiprows=True).transpose()

In [ ]:
data[2000:2020]

In [ ]:
print("Sampled text:\n{0:s}".format(sample(list(data[2000:2020]), 100, model_baseline)[0]))

In [ ]:
polish_samples = ["Zosia ze wstążki kokardę zrobił", 
                  "bocian ma skrzydła biał"]

s_ix = 1
model_sample_results = []

for i, s in enumerate(polish_samples):
    model_sample_results.append(sample(list(polish_samples[i]), 1, model_baseline))
    print("\n------------Sampled text-----------:\n{0:s}".format(model_sample_results[i][0]))

k = 10

top_k_ixs = np.argpartition(model_sample_results[s_ix][1], -k, axis=None)[-k:]
top_k_probs = model_sample_results[s_ix][1][np.argpartition(model_sample_results[s_ix][1], 
                                                       -k, axis=None)[-k:]]

print("{0:>5s} | {1:6s}".format("char", "prob"))
print("\n".join(["{0:>5s} | {1:>.5f}".format(c, p) 
                for c, p in zip([ix_to_char[ix] for ix in top_k_ixs], top_k_probs)]))

In [ ]:
sample(list(data[2000:2020])

In [ ]:
fig1, ax1 = plt.subplots()
ax1.plot(log_train[0], log_train[1])
ax1.set_xlim(0, (n_epochs * train_size))
ax1.set_xlabel("iteration")
ax1.set_ylabel("loss", color="r")
for tlbl in ax1.get_yticklabels():
    tlbl.set_color("r")
plt.legend(['training loss'], bbox_to_anchor=(1.48, 1.05), framealpha=0)
plt.tight_layout()

## Question 1 - evaluation using perplexity


Implement **perplexity** metric calculation. Perplexity is the standard measure used to evaluate language modeling performance.

Add the following code:


## Pending

Defining a test set and evaluation using perplexity measure

## possible questions (in work)

- What happens if we increase the batch size (or the training sequence)?
- Modify the network to include more layers. Go deep vs go wide.
- Back-propagation through Time? Why?
- GRU vs LSTM?
- Can we modify the code to a word level neural network model? This can also be an assignment question.
- Plot embeddings over the words, or characters
- Character level vs Word level RNN?